![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [1]:
print("Works")

Works


In [2]:
import os
import pandas as pd
import ibm_boto3
from ibm_botocore.client import Config
import torch
from getpass import getpass
import types
def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass("Please enter your api key (hit enter): ")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [3]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [40]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 800,
    "repetition_penalty": 1.7,
    'task': 'text-completion',
    'temperature': 0.9,
    'top_k': 50,
    'top_p': 0.9,
}


## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [41]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [45]:
# Import the IBM Watsonx AI Model class
from ibm_watsonx_ai.foundation_models import Model

# Initialize the IBM Watsonx AI model
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=get_credentials(),
    project_id=project_id,
    space_id=space_id
)

#lr0uM4vWyU57-E5JFdxIVmn0yoTEVbElljjOwEJ7ZG5z

Please enter your api key (hit enter): ········


In [47]:
# IBM Cloud Object Storage client setup for loading dataset
cos_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='zjce4m8BHwXXsZdoZUEeI390wl8-6Kc-mgmYDU6oDJq1',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-de.cloud-object-storage.appdomain.cloud'
)

bucket = 'allam2-donotdelete-pr-ifdnbaat2riscd'
object_key = 'data.csv'

# Get the dataset from Cloud Object Storage
body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Ensure the body is iterable so pandas can read it as a file-like object
if not hasattr(body, "__iter__"):
    def __iter__(self): return 0
    body.__iter__ = types.MethodType(__iter__, body)

# Read the Excel file
data = pd.read_csv(body)

# Display the first few rows to verify
print(data.head())

                                             Content
0  في بالي من كلام الناس كل اللي في بال الناس\nجم...
1  إنفجار..\n.\n.\nإعتقال.\n.\n.\nإنتحار..\n.\n.\...
2  جيتك ونظراتي مفاتيح الأفكار\nلي فيك نظرة غير م...
3  محتاج\nوالله عشان أحيا..\nمحتاج لي شيّ يقتل حي...
4  هذا الصباح.. إحباط. مظلم, مختنق فيه الصباح\nوأ...


In [48]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Convert the data column to a list of strings for processing
data_list = data['Content'].tolist()

# Combine data with user_input
user_input = "الليل المطر"
combined_data = data_list + [user_input]

# Fit the TfidfVectorizer on the combined data
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(combined_data)

# Get the TF-IDF vector for the last document, which is user_input
user_input_tfidf = tfidf_matrix[-1]

# Calculate cosine similarity between user_input and each document in data
similarities = cosine_similarity(user_input_tfidf, tfidf_matrix[:-1]).flatten()

# Get the indices of the top 3 most similar poems
top_3_indices = similarities.argsort()[-3:][::-1]

# Retrieve the top 3 most similar poems and their similarity scores
top_3_poems = [(data_list[i], similarities[i]) for i in top_3_indices]

# Display the results
for i, (poem, score) in enumerate(top_3_poems, 1):
    print(f"Top {i} Similar Poem (Score: {score}):\n{poem}\n")

Top 1 Similar Poem (Score: 0.10915908045782236):
شاعر تِغرد جوف صدره سكاكين
بَرْد التشرد في شوارع مدينه
أَنْحَتْ على رمل الأماكن ذِرَاعَين
وَاْحطب من أضغاث التمني غبينه
صوتي تلاوة صوت في نبرة آمين
موتي وطن سور المقابر يدينه
صليت في مفرق دعاء المعزين
دافي بلون أَغْمَاضَة الياسمينه
شفني أقول : الليل حائط محبين
شفني ألون ف الفضا لي سكينه
أركض ورى تَكّ المطر يابس العين
وأنفض غبار الفقد عن وجد طينه !!
كان العطش عصفور نبت المساكين
وأمي خشوع السجدتين الحزينه
أعلك سواك أصحاب تبغ ورياحين
في دفتر أوجاع القلم ظل أنينه
الدرب تُربه سلك شائك وتخمين
والبحر نَاسِك في شراع السفينه
شفني أجر أذيال حلمي على وين ؟!
ضايق أجوب الليل وأسكن عرينه
عالق وروحي ف الحطب بين فاسين
شف لي وطن وأغفو بحنطة حنينه
حاصل عذوق الغيم ما جاب وافين
والموت في بحة رجائي غبينه
ما هي غريبة لا أنطفى فيدي اللين
قل للورق شاعرك سافه سنينه

Top 2 Similar Poem (Score: 0.0806150334802618):
معك ليلِ طويل وقلب تاه بهاجسك مافاق.
وايقْض لك وجل شاعر خذته اردان ديمومة
تمناك الشعر ليلة نزف لك في مهب فراق
سأل سكينك البارد عن اول واخر علومه
انا يا

## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [49]:
prompt_input = f"""<<SYS>>
أنت كاتب قصائد سعودي محترف ملم بالاسلوب السعودي في الشعر.
الهدف: كتابة شعر باسلوب سعودي يتكون من ثلاثة إلى خمسة أبيات فقط، بحيث تكون الأبيات قصيرة ومباشرة، مع تجنب تكرار الأبيات أو إعادة القصيدة بأكملها بناء على العبارة المعطاة وتكون تنتهي بنفس القافية.
اكتب قصيدة جديدة ومختصرة تتكون من ثلاثة إلى خمسة أبيات فقط، بدون أي تكرار أو إعادة للقصيدة بأكملها، ثم توقف عن الكتابة بعد إتمام القصيدة.
<</SYS>>

"""

formattedQuestion = f"""[INST] 
العبارة المعطاة: "{user_input}"

هذه الأمثلة للشعر يجب أن تكون للإلهام والاستدلال فقط، ركّز على نفس المشاعر والأسلوب، ولكن لا تكرر الأبيات نفسها ولا تستخدم أي جزء منها.
- الشعر الأول: "{top_3_poems[0][0]}"
- الشعر الثاني: "{top_3_poems[1][0]}"
- الشعر الثالث: "{top_3_poems[2][0]}"
[/INST]"""

In [50]:
print(prompt_input)
print(formattedQuestion)

<<SYS>>
أنت كاتب قصائد سعودي محترف ملم بالاسلوب السعودي في الشعر.
الهدف: كتابة شعر باسلوب سعودي يتكون من ثلاثة إلى خمسة أبيات فقط، بحيث تكون الأبيات قصيرة ومباشرة، مع تجنب تكرار الأبيات أو إعادة القصيدة بأكملها بناء على العبارة المعطاة وتكون تنتهي بنفس القافية.
اكتب قصيدة جديدة ومختصرة تتكون من ثلاثة إلى خمسة أبيات فقط، بدون أي تكرار أو إعادة للقصيدة بأكملها، ثم توقف عن الكتابة بعد إتمام القصيدة.
<</SYS>>


[INST] 
العبارة المعطاة: "الليل المطر"

هذه الأمثلة للشعر يجب أن تكون للإلهام والاستدلال فقط، ركّز على نفس المشاعر والأسلوب، ولكن لا تكرر الأبيات نفسها ولا تستخدم أي جزء منها.
- الشعر الأول: "شاعر تِغرد جوف صدره سكاكين
بَرْد التشرد في شوارع مدينه
أَنْحَتْ على رمل الأماكن ذِرَاعَين
وَاْحطب من أضغاث التمني غبينه
صوتي تلاوة صوت في نبرة آمين
موتي وطن سور المقابر يدينه
صليت في مفرق دعاء المعزين
دافي بلون أَغْمَاضَة الياسمينه
شفني أقول : الليل حائط محبين
شفني ألون ف الفضا لي سكينه
أركض ورى تَكّ المطر يابس العين
وأنفض غبار الفقد عن وجد طينه !!
كان العطش عصفور نبت المساكين
وأمي خشوع السجدتي

## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [51]:
#question = input("Question: ")
#formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"{generated_response}")


 يا مطر اليل عانق روح عاشق حزين  	
نسمة هوا تجتاح قلبً بلا مرسى 
هموم واحزان تسكن وسط الضلوعين   	
والامل ضايع بوسط بحر الامنيات المرسى    


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  